In [27]:


import pandas as pd
import numpy as np

train = pd.read_csv('academic_data.csv')

test = pd.read_csv('test.csv')

sample= pd.read_csv('sample_submission.csv')

train.head()

train.drop(columns=['Course'],inplace  =True)

# test.drop(columns=['Course'],inplace = True)

train.isnull().sum()

train.shape

test.shape

test.isnull().sum()

test.head()

test.drop(columns=['id'],inplace = True)

test.shape

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = train.drop(columns=['Target'],axis = 1)

y =train['Target']

x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)

train.shape

test.shape

train.head()

test.drop(columns=['Course'],inplace = True)



from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(test)


,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,1
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,2
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,2
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,3
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,1


In [80]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier


In [94]:
from sklearn.ensemble import StackingClassifier

base_models = []

base_models.append(('xgb', XGBClassifier()))
base_models.append(('lgb', LGBMClassifier(verbose=-1,learning_rate=0.01,n_estimators=550,n_jobs=-1)))
base_models.append(('cat', CatBoostClassifier(verbose=0,iterations=1700,learning_rate=0.05)))

# base_models.append(('lr', LogisticRegression(max_iter=10000)))
# base_models.append(('dt', DecisionTreeClassifier(criterion='gini',splitter='best')))
# base_models.append(('rf', RandomForestClassifier(n_estimators=150,criterion='gini',max_depth=6)))
# base_models.append(('gbc', GradientBoostingClassifier(loss='log_loss',random_state=42,n_estimators=142,learning_rate=0.1)))


meta_model = LogisticRegression(max_iter=10000)
clfstck = StackingClassifier(estimators=base_models, final_estimator=meta_model)

clfstck.fit(x_train, y_train)
clf_y_pred = clfstck.predict(x_val)
clf_score = accuracy_score(y_val,clf_y_pred)
clf_score

0.8338342916884475

In [95]:
clf_score*100

83.38342916884474

## GradientBoostingClassifier algo

In [122]:
from sklearn.ensemble import GradientBoostingClassifier
gar = GradientBoostingClassifier(n_estimators=150,verbose=0,loss='log_loss',learning_rate=0.05,random_state=9,max_depth=5)
gar.fit(x_train,y_train)
gari = gar.predict(x_val)
accuracy_score(y_val,gari)

0.832919498170413

In [125]:
accgar = accuracy_score(y_val,gari)*100

In [126]:
accgar

83.2919498170413

In [28]:
gar.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

## catboost algo

from catboost import CatBoostClassifier

cat = CatBoostClassifier(n_estimators=1700,learning_rate=0.05)
cat.fit(x_train,y_train)
cat_acc= cat.predict(x_val)
accuracy_score(y_val,cat_acc)*100

In [68]:
catout = accuracy_score(y_val,cat_acc)*100

In [69]:
catout

83.44223732357554

In [70]:
cat.get_params()

{'learning_rate': 0.05, 'n_estimators': 1700}

## lightGBM code

import lightgbm as lgb

lll = lgb.LGBMClassifier(n_estimators=350, random_state=35,n_jobs=-1,learning_rate=0.06)

lll.fit(x_train,y_train)

outputlll = lll.predict(x_val)

accuracy_score(y_val,outputlll)*100

In [109]:
lll.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.06,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 350,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 35,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [125]:
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

In [210]:
stacking = StackingClassifier(
    estimators=[
        ('gb', gar),
        ('cat', cat)
    ],
    final_estimator=LogisticRegression(),
    cv=5  
)

In [211]:
stacking.fit(x_train,y_train)

0:	learn: 1.0445425	total: 41.2ms	remaining: 1m 10s
1:	learn: 0.9971504	total: 77.2ms	remaining: 1m 5s
2:	learn: 0.9555202	total: 108ms	remaining: 1m
3:	learn: 0.9177363	total: 139ms	remaining: 59s
4:	learn: 0.8844710	total: 170ms	remaining: 57.6s
5:	learn: 0.8546592	total: 198ms	remaining: 55.8s
6:	learn: 0.8277325	total: 228ms	remaining: 55.1s
7:	learn: 0.8029863	total: 258ms	remaining: 54.5s
8:	learn: 0.7796664	total: 287ms	remaining: 53.9s
9:	learn: 0.7589519	total: 317ms	remaining: 53.6s
10:	learn: 0.7399822	total: 347ms	remaining: 53.2s
11:	learn: 0.7222078	total: 375ms	remaining: 52.8s
12:	learn: 0.7059312	total: 405ms	remaining: 52.6s
13:	learn: 0.6913024	total: 436ms	remaining: 52.5s
14:	learn: 0.6776923	total: 467ms	remaining: 52.5s
15:	learn: 0.6654464	total: 498ms	remaining: 52.4s
16:	learn: 0.6537285	total: 527ms	remaining: 52.2s
17:	learn: 0.6428541	total: 555ms	remaining: 51.8s
18:	learn: 0.6330971	total: 585ms	remaining: 51.8s
19:	learn: 0.6236243	total: 613ms	remaining

162:	learn: 0.4507717	total: 4.61s	remaining: 43.5s
163:	learn: 0.4506283	total: 4.64s	remaining: 43.5s
164:	learn: 0.4505189	total: 4.67s	remaining: 43.5s
165:	learn: 0.4504312	total: 4.7s	remaining: 43.4s
166:	learn: 0.4503242	total: 4.73s	remaining: 43.4s
167:	learn: 0.4501878	total: 4.76s	remaining: 43.4s
168:	learn: 0.4500979	total: 4.79s	remaining: 43.4s
169:	learn: 0.4499890	total: 4.82s	remaining: 43.4s
170:	learn: 0.4498347	total: 4.86s	remaining: 43.5s
171:	learn: 0.4496193	total: 4.89s	remaining: 43.5s
172:	learn: 0.4494479	total: 4.92s	remaining: 43.4s
173:	learn: 0.4493580	total: 4.95s	remaining: 43.4s
174:	learn: 0.4492720	total: 4.98s	remaining: 43.4s
175:	learn: 0.4491283	total: 5s	remaining: 43.3s
176:	learn: 0.4490496	total: 5.04s	remaining: 43.3s
177:	learn: 0.4489449	total: 5.08s	remaining: 43.4s
178:	learn: 0.4488020	total: 5.12s	remaining: 43.5s
179:	learn: 0.4487331	total: 5.14s	remaining: 43.4s
180:	learn: 0.4485504	total: 5.17s	remaining: 43.4s
181:	learn: 0.44

323:	learn: 0.4351513	total: 8.97s	remaining: 38.1s
324:	learn: 0.4350966	total: 8.99s	remaining: 38s
325:	learn: 0.4350468	total: 9.01s	remaining: 38s
326:	learn: 0.4349937	total: 9.04s	remaining: 38s
327:	learn: 0.4348657	total: 9.06s	remaining: 37.9s
328:	learn: 0.4347895	total: 9.09s	remaining: 37.9s
329:	learn: 0.4347158	total: 9.11s	remaining: 37.8s
330:	learn: 0.4346477	total: 9.13s	remaining: 37.8s
331:	learn: 0.4345772	total: 9.16s	remaining: 37.7s
332:	learn: 0.4344604	total: 9.19s	remaining: 37.7s
333:	learn: 0.4344219	total: 9.21s	remaining: 37.7s
334:	learn: 0.4343507	total: 9.24s	remaining: 37.6s
335:	learn: 0.4342892	total: 9.26s	remaining: 37.6s
336:	learn: 0.4341435	total: 9.29s	remaining: 37.6s
337:	learn: 0.4340643	total: 9.31s	remaining: 37.5s
338:	learn: 0.4340028	total: 9.34s	remaining: 37.5s
339:	learn: 0.4339565	total: 9.36s	remaining: 37.4s
340:	learn: 0.4338864	total: 9.38s	remaining: 37.4s
341:	learn: 0.4338415	total: 9.41s	remaining: 37.4s
342:	learn: 0.4337

490:	learn: 0.4243934	total: 13.1s	remaining: 32.3s
491:	learn: 0.4243143	total: 13.1s	remaining: 32.2s
492:	learn: 0.4242307	total: 13.2s	remaining: 32.2s
493:	learn: 0.4241513	total: 13.2s	remaining: 32.2s
494:	learn: 0.4240704	total: 13.2s	remaining: 32.2s
495:	learn: 0.4239961	total: 13.2s	remaining: 32.1s
496:	learn: 0.4239702	total: 13.3s	remaining: 32.1s
497:	learn: 0.4239219	total: 13.3s	remaining: 32.1s
498:	learn: 0.4238700	total: 13.3s	remaining: 32.1s
499:	learn: 0.4238346	total: 13.3s	remaining: 32s
500:	learn: 0.4237873	total: 13.4s	remaining: 32s
501:	learn: 0.4237579	total: 13.4s	remaining: 32s
502:	learn: 0.4236995	total: 13.4s	remaining: 32s
503:	learn: 0.4236676	total: 13.5s	remaining: 31.9s
504:	learn: 0.4235874	total: 13.5s	remaining: 31.9s
505:	learn: 0.4235424	total: 13.5s	remaining: 31.9s
506:	learn: 0.4235008	total: 13.5s	remaining: 31.8s
507:	learn: 0.4234575	total: 13.6s	remaining: 31.8s
508:	learn: 0.4234159	total: 13.6s	remaining: 31.8s
509:	learn: 0.423375

656:	learn: 0.4162760	total: 17.5s	remaining: 27.7s
657:	learn: 0.4162185	total: 17.5s	remaining: 27.7s
658:	learn: 0.4161712	total: 17.5s	remaining: 27.7s
659:	learn: 0.4161091	total: 17.5s	remaining: 27.6s
660:	learn: 0.4160638	total: 17.6s	remaining: 27.6s
661:	learn: 0.4159844	total: 17.6s	remaining: 27.6s
662:	learn: 0.4159302	total: 17.6s	remaining: 27.5s
663:	learn: 0.4158982	total: 17.6s	remaining: 27.5s
664:	learn: 0.4158556	total: 17.7s	remaining: 27.5s
665:	learn: 0.4158177	total: 17.7s	remaining: 27.5s
666:	learn: 0.4157851	total: 17.7s	remaining: 27.4s
667:	learn: 0.4157610	total: 17.7s	remaining: 27.4s
668:	learn: 0.4157288	total: 17.8s	remaining: 27.4s
669:	learn: 0.4157024	total: 17.8s	remaining: 27.4s
670:	learn: 0.4156537	total: 17.8s	remaining: 27.3s
671:	learn: 0.4156153	total: 17.8s	remaining: 27.3s
672:	learn: 0.4155612	total: 17.9s	remaining: 27.3s
673:	learn: 0.4155131	total: 17.9s	remaining: 27.3s
674:	learn: 0.4154661	total: 17.9s	remaining: 27.2s
675:	learn: 

...

1175:	learn: 0.3910341	total: 28.1s	remaining: 12.5s
1176:	learn: 0.3909650	total: 28.1s	remaining: 12.5s
1177:	learn: 0.3909329	total: 28.2s	remaining: 12.5s
1178:	learn: 0.3909137	total: 28.2s	remaining: 12.5s
1179:	learn: 0.3908394	total: 28.2s	remaining: 12.4s
1180:	learn: 0.3907663	total: 28.2s	remaining: 12.4s
1181:	learn: 0.3907287	total: 28.3s	remaining: 12.4s
1182:	learn: 0.3906925	total: 28.3s	remaining: 12.4s
1183:	learn: 0.3906631	total: 28.3s	remaining: 12.3s
1184:	learn: 0.3906176	total: 28.3s	remaining: 12.3s
1185:	learn: 0.3905998	total: 28.4s	remaining: 12.3s
1186:	learn: 0.3905382	total: 28.4s	remaining: 12.3s
1187:	learn: 0.3904779	total: 28.4s	remaining: 12.2s
1188:	learn: 0.3904584	total: 28.4s	remaining: 12.2s
1189:	learn: 0.3904098	total: 28.5s	remaining: 12.2s
1190:	learn: 0.3903795	total: 28.5s	remaining: 12.2s
1191:	learn: 0.3903061	total: 28.5s	remaining: 12.2s
1192:	learn: 0.3902629	total: 28.5s	remaining: 12.1s
1193:	learn: 0.3902281	total: 28.6s	remaining:

1331:	learn: 0.3842266	total: 31.9s	remaining: 8.81s
1332:	learn: 0.3841967	total: 31.9s	remaining: 8.79s
1333:	learn: 0.3841705	total: 31.9s	remaining: 8.76s
1334:	learn: 0.3841537	total: 32s	remaining: 8.74s
1335:	learn: 0.3841082	total: 32s	remaining: 8.72s
1336:	learn: 0.3840788	total: 32s	remaining: 8.69s
1337:	learn: 0.3840159	total: 32s	remaining: 8.67s
1338:	learn: 0.3839966	total: 32.1s	remaining: 8.64s
1339:	learn: 0.3839549	total: 32.1s	remaining: 8.62s
1340:	learn: 0.3839371	total: 32.1s	remaining: 8.6s
1341:	learn: 0.3839129	total: 32.1s	remaining: 8.57s
1342:	learn: 0.3838950	total: 32.2s	remaining: 8.55s
1343:	learn: 0.3838167	total: 32.2s	remaining: 8.52s
1344:	learn: 0.3837997	total: 32.2s	remaining: 8.5s
1345:	learn: 0.3837562	total: 32.2s	remaining: 8.48s
1346:	learn: 0.3837223	total: 32.3s	remaining: 8.45s
1347:	learn: 0.3836386	total: 32.3s	remaining: 8.43s
1348:	learn: 0.3836216	total: 32.3s	remaining: 8.4s
1349:	learn: 0.3835786	total: 32.3s	remaining: 8.38s
1350

1491:	learn: 0.3774732	total: 35.8s	remaining: 4.98s
1492:	learn: 0.3774356	total: 35.8s	remaining: 4.96s
1493:	learn: 0.3774086	total: 35.8s	remaining: 4.94s
1494:	learn: 0.3773870	total: 35.8s	remaining: 4.91s
1495:	learn: 0.3773669	total: 35.9s	remaining: 4.89s
1496:	learn: 0.3773417	total: 35.9s	remaining: 4.87s
1497:	learn: 0.3772974	total: 35.9s	remaining: 4.84s
1498:	learn: 0.3772635	total: 35.9s	remaining: 4.82s
1499:	learn: 0.3772447	total: 36s	remaining: 4.79s
1500:	learn: 0.3771525	total: 36s	remaining: 4.77s
1501:	learn: 0.3770995	total: 36s	remaining: 4.75s
1502:	learn: 0.3770777	total: 36s	remaining: 4.72s
1503:	learn: 0.3769804	total: 36.1s	remaining: 4.7s
1504:	learn: 0.3769606	total: 36.1s	remaining: 4.67s
1505:	learn: 0.3769316	total: 36.1s	remaining: 4.65s
1506:	learn: 0.3768726	total: 36.1s	remaining: 4.63s
1507:	learn: 0.3768039	total: 36.2s	remaining: 4.6s
1508:	learn: 0.3767714	total: 36.2s	remaining: 4.58s
1509:	learn: 0.3767582	total: 36.2s	remaining: 4.55s
151

1651:	learn: 0.3713415	total: 39.6s	remaining: 1.15s
1652:	learn: 0.3712589	total: 39.6s	remaining: 1.13s
1653:	learn: 0.3712479	total: 39.6s	remaining: 1.1s
1654:	learn: 0.3712155	total: 39.7s	remaining: 1.08s
1655:	learn: 0.3711658	total: 39.7s	remaining: 1.05s
1656:	learn: 0.3711031	total: 39.7s	remaining: 1.03s
1657:	learn: 0.3710628	total: 39.7s	remaining: 1.01s
1658:	learn: 0.3710282	total: 39.8s	remaining: 983ms
1659:	learn: 0.3709584	total: 39.8s	remaining: 959ms
1660:	learn: 0.3709221	total: 39.8s	remaining: 935ms
1661:	learn: 0.3708418	total: 39.8s	remaining: 911ms
1662:	learn: 0.3707918	total: 39.9s	remaining: 887ms
1663:	learn: 0.3707660	total: 39.9s	remaining: 863ms
1664:	learn: 0.3707342	total: 39.9s	remaining: 839ms
1665:	learn: 0.3707140	total: 39.9s	remaining: 815ms
1666:	learn: 0.3706906	total: 40s	remaining: 791ms
1667:	learn: 0.3706068	total: 40s	remaining: 767ms
1668:	learn: 0.3705931	total: 40s	remaining: 743ms
1669:	learn: 0.3705678	total: 40s	remaining: 719ms
16

StackingClassifier(cv=5,
                   estimators=[('gb',
                                GradientBoostingClassifier(learning_rate=0.05,
                                                           max_depth=5,
                                                           n_estimators=150,
                                                           random_state=9)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x0000026A485ADCD0>)],
                   final_estimator=LogisticRegression())

In [212]:
stack = stacking.predict(x_val)

In [213]:
accuracy_score(y_val,stack)*100

83.4553058024046

## voting class

In [39]:
from sklearn.ensemble import VotingClassifier

In [114]:
voting_clf = VotingClassifier(
    estimators=[('gb', gar), ('cat', cat), ('light',lll)],
    voting='soft'  
)

In [115]:
voting_clf.fit(x_train,y_train)

0:	learn: 1.0445425	total: 43.2ms	remaining: 1m 13s
1:	learn: 0.9971504	total: 81.7ms	remaining: 1m 9s
2:	learn: 0.9555202	total: 117ms	remaining: 1m 6s
3:	learn: 0.9177363	total: 152ms	remaining: 1m 4s
4:	learn: 0.8844710	total: 187ms	remaining: 1m 3s
5:	learn: 0.8546592	total: 219ms	remaining: 1m 1s
6:	learn: 0.8277325	total: 258ms	remaining: 1m 2s
7:	learn: 0.8029863	total: 298ms	remaining: 1m 3s
8:	learn: 0.7796664	total: 332ms	remaining: 1m 2s
9:	learn: 0.7589519	total: 360ms	remaining: 1m
10:	learn: 0.7399822	total: 387ms	remaining: 59.4s
11:	learn: 0.7222078	total: 414ms	remaining: 58.2s
12:	learn: 0.7059312	total: 440ms	remaining: 57.1s
13:	learn: 0.6913024	total: 467ms	remaining: 56.3s
14:	learn: 0.6776923	total: 498ms	remaining: 55.9s
15:	learn: 0.6654464	total: 528ms	remaining: 55.5s
16:	learn: 0.6537285	total: 557ms	remaining: 55.1s
17:	learn: 0.6428541	total: 585ms	remaining: 54.6s
18:	learn: 0.6330971	total: 615ms	remaining: 54.4s
19:	learn: 0.6236243	total: 644ms	remaini

161:	learn: 0.4509436	total: 4.35s	remaining: 41.3s
162:	learn: 0.4507717	total: 4.38s	remaining: 41.3s
163:	learn: 0.4506283	total: 4.4s	remaining: 41.2s
164:	learn: 0.4505189	total: 4.42s	remaining: 41.1s
165:	learn: 0.4504312	total: 4.45s	remaining: 41.1s
166:	learn: 0.4503242	total: 4.47s	remaining: 41.1s
167:	learn: 0.4501878	total: 4.5s	remaining: 41s
168:	learn: 0.4500979	total: 4.52s	remaining: 40.9s
169:	learn: 0.4499890	total: 4.54s	remaining: 40.9s
170:	learn: 0.4498347	total: 4.57s	remaining: 40.9s
171:	learn: 0.4496193	total: 4.59s	remaining: 40.8s
172:	learn: 0.4494479	total: 4.62s	remaining: 40.8s
173:	learn: 0.4493580	total: 4.64s	remaining: 40.7s
174:	learn: 0.4492720	total: 4.66s	remaining: 40.6s
175:	learn: 0.4491283	total: 4.69s	remaining: 40.6s
176:	learn: 0.4490496	total: 4.71s	remaining: 40.5s
177:	learn: 0.4489449	total: 4.74s	remaining: 40.5s
178:	learn: 0.4488020	total: 4.76s	remaining: 40.5s
179:	learn: 0.4487331	total: 4.79s	remaining: 40.4s
180:	learn: 0.44

327:	learn: 0.4348657	total: 8.49s	remaining: 35.5s
328:	learn: 0.4347895	total: 8.52s	remaining: 35.5s
329:	learn: 0.4347158	total: 8.55s	remaining: 35.5s
330:	learn: 0.4346477	total: 8.57s	remaining: 35.5s
331:	learn: 0.4345772	total: 8.6s	remaining: 35.4s
332:	learn: 0.4344604	total: 8.63s	remaining: 35.4s
333:	learn: 0.4344219	total: 8.65s	remaining: 35.4s
334:	learn: 0.4343507	total: 8.68s	remaining: 35.4s
335:	learn: 0.4342892	total: 8.71s	remaining: 35.4s
336:	learn: 0.4341435	total: 8.95s	remaining: 36.2s
337:	learn: 0.4340643	total: 9.02s	remaining: 36.3s
338:	learn: 0.4340028	total: 9.07s	remaining: 36.4s
339:	learn: 0.4339565	total: 9.15s	remaining: 36.6s
340:	learn: 0.4338864	total: 9.2s	remaining: 36.6s
341:	learn: 0.4338415	total: 9.23s	remaining: 36.6s
342:	learn: 0.4337774	total: 9.26s	remaining: 36.6s
343:	learn: 0.4336884	total: 9.29s	remaining: 36.6s
344:	learn: 0.4335993	total: 9.32s	remaining: 36.6s
345:	learn: 0.4335481	total: 9.34s	remaining: 36.6s
346:	learn: 0.

491:	learn: 0.4243143	total: 13.3s	remaining: 32.6s
492:	learn: 0.4242307	total: 13.3s	remaining: 32.6s
493:	learn: 0.4241513	total: 13.3s	remaining: 32.5s
494:	learn: 0.4240704	total: 13.4s	remaining: 32.5s
495:	learn: 0.4239961	total: 13.4s	remaining: 32.5s
496:	learn: 0.4239702	total: 13.4s	remaining: 32.5s
497:	learn: 0.4239219	total: 13.4s	remaining: 32.4s
498:	learn: 0.4238700	total: 13.5s	remaining: 32.4s
499:	learn: 0.4238346	total: 13.5s	remaining: 32.4s
500:	learn: 0.4237873	total: 13.5s	remaining: 32.4s
501:	learn: 0.4237579	total: 13.5s	remaining: 32.3s
502:	learn: 0.4236995	total: 13.6s	remaining: 32.3s
503:	learn: 0.4236676	total: 13.6s	remaining: 32.3s
504:	learn: 0.4235874	total: 13.6s	remaining: 32.3s
505:	learn: 0.4235424	total: 13.7s	remaining: 32.2s
506:	learn: 0.4235008	total: 13.7s	remaining: 32.3s
507:	learn: 0.4234575	total: 13.7s	remaining: 32.2s
508:	learn: 0.4234159	total: 13.8s	remaining: 32.2s
509:	learn: 0.4233751	total: 13.8s	remaining: 32.1s
510:	learn: 

650:	learn: 0.4165713	total: 17.7s	remaining: 28.5s
651:	learn: 0.4164962	total: 17.7s	remaining: 28.5s
652:	learn: 0.4164406	total: 17.8s	remaining: 28.5s
653:	learn: 0.4163938	total: 17.8s	remaining: 28.4s
654:	learn: 0.4163502	total: 17.8s	remaining: 28.4s
655:	learn: 0.4163205	total: 17.8s	remaining: 28.4s
656:	learn: 0.4162760	total: 17.9s	remaining: 28.4s
657:	learn: 0.4162185	total: 17.9s	remaining: 28.3s
658:	learn: 0.4161712	total: 17.9s	remaining: 28.3s
659:	learn: 0.4161091	total: 17.9s	remaining: 28.3s
660:	learn: 0.4160638	total: 18s	remaining: 28.2s
661:	learn: 0.4159844	total: 18s	remaining: 28.2s
662:	learn: 0.4159302	total: 18s	remaining: 28.2s
663:	learn: 0.4158982	total: 18s	remaining: 28.1s
664:	learn: 0.4158556	total: 18.1s	remaining: 28.1s
665:	learn: 0.4158177	total: 18.1s	remaining: 28.1s
666:	learn: 0.4157851	total: 18.1s	remaining: 28.1s
667:	learn: 0.4157610	total: 18.1s	remaining: 28s
668:	learn: 0.4157288	total: 18.2s	remaining: 28s
669:	learn: 0.4157024	to

...

1125:	learn: 0.3975166	total: 29.6s	remaining: 15.1s
1126:	learn: 0.3974833	total: 29.6s	remaining: 15.1s
1127:	learn: 0.3974315	total: 29.7s	remaining: 15s
1128:	learn: 0.3973993	total: 29.7s	remaining: 15s
1129:	learn: 0.3973727	total: 29.7s	remaining: 15s
1130:	learn: 0.3973341	total: 29.7s	remaining: 15s
1131:	learn: 0.3972643	total: 29.8s	remaining: 14.9s
1132:	learn: 0.3972439	total: 29.8s	remaining: 14.9s
1133:	learn: 0.3972092	total: 29.8s	remaining: 14.9s
1134:	learn: 0.3971774	total: 29.8s	remaining: 14.8s
1135:	learn: 0.3971294	total: 29.9s	remaining: 14.8s
1136:	learn: 0.3970592	total: 29.9s	remaining: 14.8s
1137:	learn: 0.3970382	total: 29.9s	remaining: 14.8s
1138:	learn: 0.3970017	total: 29.9s	remaining: 14.7s
1139:	learn: 0.3969604	total: 30s	remaining: 14.7s
1140:	learn: 0.3969495	total: 30s	remaining: 14.7s
1141:	learn: 0.3969252	total: 30s	remaining: 14.7s
1142:	learn: 0.3968862	total: 30s	remaining: 14.6s
1143:	learn: 0.3968484	total: 30.1s	remaining: 14.6s
1144:	lea

1282:	learn: 0.3921021	total: 33.6s	remaining: 10.9s
1283:	learn: 0.3920810	total: 33.6s	remaining: 10.9s
1284:	learn: 0.3920513	total: 33.6s	remaining: 10.9s
1285:	learn: 0.3920056	total: 33.6s	remaining: 10.8s
1286:	learn: 0.3919442	total: 33.7s	remaining: 10.8s
1287:	learn: 0.3919375	total: 33.7s	remaining: 10.8s
1288:	learn: 0.3918764	total: 33.7s	remaining: 10.7s
1289:	learn: 0.3918502	total: 33.7s	remaining: 10.7s
1290:	learn: 0.3918223	total: 33.8s	remaining: 10.7s
1291:	learn: 0.3917960	total: 33.8s	remaining: 10.7s
1292:	learn: 0.3917209	total: 33.8s	remaining: 10.6s
1293:	learn: 0.3916831	total: 33.8s	remaining: 10.6s
1294:	learn: 0.3916500	total: 33.9s	remaining: 10.6s
1295:	learn: 0.3916300	total: 33.9s	remaining: 10.6s
1296:	learn: 0.3915808	total: 33.9s	remaining: 10.5s
1297:	learn: 0.3915595	total: 33.9s	remaining: 10.5s
1298:	learn: 0.3915213	total: 34s	remaining: 10.5s
1299:	learn: 0.3914961	total: 34s	remaining: 10.5s
1300:	learn: 0.3914503	total: 34s	remaining: 10.4s

1443:	learn: 0.3863083	total: 37.7s	remaining: 6.67s
1444:	learn: 0.3862644	total: 37.7s	remaining: 6.65s
1445:	learn: 0.3862209	total: 37.7s	remaining: 6.62s
1446:	learn: 0.3861704	total: 37.7s	remaining: 6.6s
1447:	learn: 0.3861304	total: 37.8s	remaining: 6.57s
1448:	learn: 0.3860886	total: 37.8s	remaining: 6.54s
1449:	learn: 0.3860343	total: 37.8s	remaining: 6.52s
1450:	learn: 0.3860024	total: 37.8s	remaining: 6.49s
1451:	learn: 0.3859659	total: 37.9s	remaining: 6.47s
1452:	learn: 0.3859067	total: 37.9s	remaining: 6.44s
1453:	learn: 0.3858448	total: 37.9s	remaining: 6.42s
1454:	learn: 0.3857964	total: 37.9s	remaining: 6.39s
1455:	learn: 0.3857866	total: 38s	remaining: 6.36s
1456:	learn: 0.3857479	total: 38s	remaining: 6.34s
1457:	learn: 0.3857315	total: 38s	remaining: 6.31s
1458:	learn: 0.3856823	total: 38s	remaining: 6.28s
1459:	learn: 0.3856378	total: 38.1s	remaining: 6.26s
1460:	learn: 0.3855915	total: 38.1s	remaining: 6.23s
1461:	learn: 0.3855325	total: 38.1s	remaining: 6.21s
14

1605:	learn: 0.3809248	total: 41.8s	remaining: 2.44s
1606:	learn: 0.3809057	total: 41.8s	remaining: 2.42s
1607:	learn: 0.3808610	total: 41.8s	remaining: 2.39s
1608:	learn: 0.3808234	total: 41.9s	remaining: 2.37s
1609:	learn: 0.3807948	total: 41.9s	remaining: 2.34s
1610:	learn: 0.3807655	total: 41.9s	remaining: 2.31s
1611:	learn: 0.3807478	total: 41.9s	remaining: 2.29s
1612:	learn: 0.3806989	total: 42s	remaining: 2.26s
1613:	learn: 0.3806789	total: 42s	remaining: 2.24s
1614:	learn: 0.3806675	total: 42s	remaining: 2.21s
1615:	learn: 0.3806151	total: 42s	remaining: 2.19s
1616:	learn: 0.3805537	total: 42.1s	remaining: 2.16s
1617:	learn: 0.3805018	total: 42.1s	remaining: 2.13s
1618:	learn: 0.3804580	total: 42.1s	remaining: 2.11s
1619:	learn: 0.3804223	total: 42.1s	remaining: 2.08s
1620:	learn: 0.3803607	total: 42.2s	remaining: 2.06s
1621:	learn: 0.3803401	total: 42.2s	remaining: 2.03s
1622:	learn: 0.3803061	total: 42.2s	remaining: 2s
1623:	learn: 0.3802731	total: 42.2s	remaining: 1.98s
1624

VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(learning_rate=0.05,
                                                         max_depth=5,
                                                         n_estimators=150,
                                                         random_state=9)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x0000026A485ADCD0>),
                             ('light',
                              LGBMClassifier(learning_rate=0.06,
                                             n_estimators=200, n_jobs=-1,
                                             random_state=35))],
                 voting='soft')

In [127]:
gar.predict(x_val)

array([1, 2, 3, ..., 1, 1, 1], dtype=int64)

In [128]:
accuracy_score(y_val,gar.predict(x_val))*100

83.2919498170413

In [129]:
predicationvoting= gar.predict(x_test)
predicationvoting
sample['Target'] = predicationvoting
catmap = {1:'Graduate',2:'Dropout',3:'Enrolled'}
sample['Target'] = sample['Target'].map(catmap)
sample.to_csv('submission.csv',index = None)
sample


